<a href="https://colab.research.google.com/github/julianafalves/ic/blob/master/Extracao_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados Principais

In [ ]:
import pandas as pd
import threading 

diag = pd.read_csv('/content/drive/My Drive/ic/ADNI1_Annual_2_Yr_1.5T_8_02_2019.csv',usecols=["Subject","Group"]) #diagnostico dos individuos   
diag_ids = diag['Subject'].values.tolist() #identidades relacionados ao seu diagnostico
diag_results = diag['Group'].values.tolist() #diagnosticos 

df1 = pd.read_csv('/content/drive/My Drive/ic/Dados Descompactados adni/adni_gwas_v2/adni_gwas_v2_set01/002_S_0295.csv')
#df2= pd.read_csv('/content/drive/My Drive/ic/Dados Descompactados adni/adni_gwas_v2/adni_gwas_v2_set1/002_S_0413.csv')

df1 =df1[df1['SNP Name'].str.startswith('rs')]
#lista = df1_filtrado['SNP Name'].values.tolist()

#roi = diag.drop_duplicates('Subject',keep='first')
df1[['SNP Name', 'Allele1 - AB','Allele2 - AB']]

,SNP Name,Allele1 - AB,Allele2 - AB
22080,rs1000000,B,B
22081,rs10000010,A,B
22082,rs10000023,A,B
22083,rs10000030,B,B
22084,rs10000041,A,A
...,...,...,...
620896,rs9999944,A,B
620897,rs999995,B,B
620898,rs9999963,A,A
620899,rs9999966,B,B


In [ ]:
import os 
folders = os.listdir('/content/drive/My Drive/ic/Dados Descompactados adni/adni_gwas_v2')
folders.sort()
folders

['adni_gwas_v2_set01',
 'adni_gwas_v2_set02',
 'adni_gwas_v2_set03',
 'adni_gwas_v2_set04',
 'adni_gwas_v2_set05',
 'adni_gwas_v2_set06',
 'adni_gwas_v2_set07',
 'adni_gwas_v2_set08',
 'adni_gwas_v2_set09',
 'adni_gwas_v2_set10',
 'adni_gwas_v2_set11']

In [ ]:
genes = ["APOE","BIN1","CLU","ABCA7","CR1","PICALM","MS4A6A","CD33","MS4A4E","CD2AP"] 


# Funções que vão ser usadas em Thread




In [ ]:
##
def Filtragem(arg):
  arg =arg[arg['SNP Name'].str.startswith('rs')]
  return arg

In [ ]:
##
def Dicionario(arq):
  lista =arq['SNP Name'].values.tolist()
  print(len(lista))

In [ ]:
xlsx=pd.read_csv('/content/drive/My Drive/ic/NomesSnpsEAlelos.csv') #leitura do com os snps e seus respectivos genes
xlsx = xlsx.transpose()
xlsx

,0,1
Unnamed: 0,Name,Alleles
0,rs7412,SNV:C>T\n
1,rs405509,SNV:T>G\n
2,rs429358,SNV:T>C\n
3,rs769452,SNV:T>C\n
...,...,...
147312,rs796829949,"DELINS:ATTTA>-,ATTTAATTTA\n"
147313,rs1416914868,"DELINS:AATT>-,AATTAATT,AATTAATTAATT,AATTAATTAA..."
147314,rs1400255377,"DELINS:TT>-,TTT\n"
147315,rs1491287923,"DELINS:AT>-,ATAT,ATATAT,ATATATAT,ATATATATATAT,..."


In [ ]:
def LeituraArquivos(folders):
  result = pd.DataFrame()
  for folder in folders:
    #file = list()
    file = os.listdir(f'/content/drive/My Drive/ic/Dados Descompactados adni/adni_gwas_v2/{folder}') #procura mais profundamente todos os .csv em todas as pastas indicadas
    file.sort()
    for id in file: #pego cada um dos cerca de 200 arquivos
        csv = pd.read_csv(f'/content/drive/My Drive/ic/Dados Descompactados adni//adni_gwas_v2/{folder}/{id}',usecols=["SNP Name","Allele1 - AB","Allele2 - AB"]) 
        csv = Filtragem(csv)
        #print(csv)

        meu_indexador = csv["SNP Name"].isin(xlsx[0])
       
        resultado_sem_comparar = csv[meu_indexador].transpose()
        resultado_sem_comparar.columns = range(resultado_sem_comparar.shape[1])

        result = pd.concat([result,compare(resultado_sem_comparar,id)])
        #print(result)´
  print('cabou')
  return pd.get_dummies(result, prefix=result.columns) #one-hot-encoding

In [ ]:
#Compara allele1-AB com allele2-AB
import numpy as np

def compare(dataframe,individuo):
  result = pd.Series(np.full(dataframe.columns.size,3),dtype='int64',name=f'{individuo}'.replace('.csv',''))

  result[(dataframe.iloc[1,:]==dataframe.iloc[2,:]) & (dataframe.iloc[1,:]=='A')] = 1
  result[(dataframe.iloc[1,:]==dataframe.iloc[2,:]) & (dataframe.iloc[1,:]=='B')] = 2
  result = result.to_frame()
  

  result = pd.concat([dataframe,result.transpose()]).drop(['Allele1 - AB','Allele2 - AB'])
  result.columns = result.iloc[0]
  result= result.drop('SNP Name')
  
  
  return result 

In [ ]:
LeituraArquivos(folders[0:1])

KeyboardInterrupt: ignored

In [ ]:

#df1 = threading.Thread(target = lambda q, arg1: q.put(LeituraArquivos(arg1)) ,args=(que,folders[0:(int(len(folders)/2))],))
#df2 = threading.Thread(target = LeituraArquivos, args=(folders[int(len(folders)/2):len(folders)],))
#df1.start()
#df2.start()
#t.join()
#result = que.get()


In [ ]:
import concurrent.futures 
with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(LeituraArquivos,folders[0:(int(len(folders)/2))])
    return_value = future.result()

with concurrent.futures.ThreadPoolExecutor() as executor:
    future2 = executor.submit(LeituraArquivos,folders[int(len(folders)/2):len(folders)])
    return_value2 = future2.result()

cabou
cabou


In [ ]:
print(return_value)

            rs10127904_1  rs10127904_2  ...  rs9473121_3  rs988337_1
002_S_0295             0             0  ...            1           1
002_S_0413             1             0  ...            1           1
002_S_0559             0             0  ...            0           1
002_S_0619             1             0  ...            0           1
002_S_0685             1             0  ...            1           1
...                  ...           ...  ...          ...         ...
037_S_0588             0             0  ...            1           1
037_S_0627             0             0  ...            0           1
037_S_1078             1             0  ...            0           1
037_S_1225             0             0  ...            1           1
037_S_1421             1             0  ...            0           1

[368 rows x 353 columns]


In [ ]:
print(return_value2)

            rs10127904_1  rs10127904_2  ...  rs9473121_3  rs988337_1
041_S_0125             1             0  ...            0           1
041_S_0262             0             0  ...            1           1
041_S_0282             1             0  ...            0           1
041_S_0314             1             0  ...            0           1
041_S_0407             0             0  ...            0           1
...                  ...           ...  ...          ...         ...
941_S_1202             0             0  ...            0           1
941_S_1203             1             0  ...            1           1
941_S_1295             0             0  ...            0           1
941_S_1311             0             0  ...            0           1
941_S_1363             1             0  ...            1           1

[389 rows x 354 columns]


In [ ]:
final_to_csv = pd.concat([return_value,return_value2])

In [ ]:
final_to_csv.to_csv('/content/drive/My Drive/ic/extracao_atributes.csv')